# Нейронные сети


<hr>

С.Ю. Папулин (papulin.study@yandex.ru)

### Содержание

- [Классификация](#Классификация)
    - [Набор данных](#Набор-данных)
    - [Логистическая регрессия](#Логистическая-регрессия)
    - [Многослойная нейронная сеть в Sklearn](#Многослойная-нейронная-сеть-в-Sklearn)
    - [Многослойная нейронная сеть в Keras](#Многослойная-нейронная-сеть-в-Keras)
    - [Выбор гиперпараметров с GridSearchCV](#Выбор-гиперпараметров-с-GridSearchCV)

Подключение библиотек:

In [ ]:
import numpy as np

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split, GridSearchCV
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

## Классификация

In [ ]:
RANDOM_STATE = 100

### Набор данных

In [ ]:
# Загрузка исходных данных
digits = datasets.load_digits()

X = digits.images.reshape(len(digits.images), -1)
y = digits.target

print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

# Замечание: можно уменьшить объем памяти и нормализовать данные
# X = X.astype("float32") / 16

IMAGE_INDX = 20

# Отображение одного изображения
print("Image:")
plt.figure(figsize=[4, 4])
plt.imshow(digits.images[IMAGE_INDX])
plt.show()

In [ ]:
# Разделение данных на обучающее и тестовое множества
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=RANDOM_STATE)

### Логистическая регрессия

Ранее уже рассматривали распознавание цифр с использованием логистической регрессии. Ниже приведен пример кода.

In [ ]:
model = LogisticRegression(C=float("inf"), 
                           multi_class="multinomial", 
                           solver="newton-cg", 
                           max_iter=200, 
                           random_state=RANDOM_STATE)
model.fit(X_train, y_train)

# Оценка качества модели
train_error = model.score(X_train, y_train)
test_error = model.score(X_test, y_test)

print("Train Accuracy:", train_error)
print("Test Accuracy:", test_error)

### Многослойная нейронная сеть в `Sklearn`

Полносвязная нейронная сеть (мultilayer perceptrons/fully connected network)

In [ ]:
# Формирование проверочного множества из исходного обучающего
X_train_, X_val, y_train_, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=RANDOM_STATE)

In [ ]:
model = MLPClassifier(hidden_layer_sizes=(80, 10),
                      solver="adam",
                      activation="relu",
                      max_iter=20,  # количество эпох
                      batch_size=5,
                      random_state=RANDOM_STATE)

model.fit(X_train, y_train)

print("Model parameters:")
for param, value in model.get_params().items():
    print("\t{} = {}".format(param, value))

# Оценка качества распознавания (доля правильных классификаций)
train_error = model.score(X_train, y_train)
test_error = model.score(X_test, y_test)
print("\nTrain Accuracy:", train_error)
print("Test Accuracy:", test_error)

# Веса модели
print("\nShape of weights and biases:")
print("\tWeights of layer 1:\t", model.coefs_[0].shape)
print("\tBaises of layer 1:\t", model.intercepts_[0].shape)
print("\tWeights of layer 2:\t", model.coefs_[1].shape)
print("\tBaises of layer 2:\t", model.intercepts_[1].shape)

In [ ]:
IMAGE_INDX = 20
sample_image_input = digits["images"][IMAGE_INDX].reshape(1, -1)

# Предсказание и отображение цифры
print("Target value:", digits.target[IMAGE_INDX])
print("Probabilities:", model.predict_proba(sample_image_input))
print("Predicted value:", model.predict(sample_image_input))
plt.imshow(digits.images[IMAGE_INDX])
plt.show()

In [ ]:
# TODO: select the number of epochs

### Многослойная нейронная сеть в `Keras`

`pip install tensorflow==2.5.0`

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical

In [ ]:
def build_model(hidden_neurons=80):
    """Построение нейронной сети."""
    
    # Топология
    model = models.Sequential()
    model.add(layers.Dense(hidden_neurons, activation="relu", input_shape=(8*8,)))
    model.add(layers.Dense(10, activation="softmax"))
    # Параметры обучения
    model.compile(optimizer="adam",
                  loss="categorical_crossentropy",
                  metrics=['accuracy'])
    return model


def build_model_alt(hidden_neurons=80):
    """Альтернативный подход к построению нейронной сети."""
    
    # Топология
    inputs = layers.Input(shape=(8*8,))
    hidden_layer = layers.Dense(hidden_neurons, activation="relu")(inputs)
    outputs = layers.Dense(10, activation="softmax")(hidden_layer)
    model = models.Model(inputs=inputs, outputs=outputs)
    # Параметры обучения
    model.compile(optimizer="adam",
                  loss="categorical_crossentropy",
                  metrics=['accuracy'])
    return model

In [ ]:
# Максимальное количество эпох
MAX_EPOCHS = 20

# Преобразование целевых значений в бинарный вектор (one-hot encoding)
Y_train_ = to_categorical(y_train_)
Y_val = to_categorical(y_val)    

# Построение модели
model = build_model_alt()

# Обучение
train_history = model.fit(X_train_, Y_train_, 
                          epochs=MAX_EPOCHS, 
                          batch_size=5,
                          validation_data=(X_val, Y_val),
                          verbose=0)

# Построение графиков ошибок обучения
plt.figure(figsize=[14, 4])

epochs = np.arange(1, len(train_history.history["loss"])+1)

plt.subplot(1,2,1)  # кросс-энтропия
plt.plot(epochs, train_history.history["loss"], "-og", label="train")
plt.plot(epochs, train_history.history["val_loss"], "-o", color="orange", label="val")
plt.xlabel("epochs")
plt.ylabel("loss")
plt.grid(True)
plt.legend()

plt.subplot(1,2,2)  # доля правильных классификаций
plt.plot(epochs, train_history.history["accuracy"], "-og", label="train")
plt.plot(epochs, train_history.history["val_accuracy"], "-o", color="orange", label="val")
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.grid(True)
plt.legend()

plt.show()

In [ ]:
# Выбираем 10 эпох и заново обучаем сеть на всём обучающем множестве
best_num_epochs = 10

# Преобразование целевых значений в бинарный вектор (one-hot encoding)
Y_train = to_categorical(y_train)
Y_test = to_categorical(y_test)

# Построение модели
model = build_model()

# Обучение
train_history = model.fit(X_train, Y_train,
                          epochs=best_num_epochs, 
                          batch_size=5,
                          verbose=0)

# Оценка качества модели
_, train_error = model.evaluate(X_train, Y_train)
_, test_error = model.evaluate(X_test, Y_test)

print("Train Accuracy:\t", train_error)
print("Test Accuracy:\t", test_error)

# Веса модели
weights = model.get_weights()

print("\nShape of weights and biases:")
print("\tWeights of layer 1:\t", weights[0].shape)
print("\tBaises of layer 1:\t", weights[1].shape)
print("\tWeights of layer 2:\t", weights[2].shape)
print("\tBaises of layer 2:\t", weights[3].shape)

In [ ]:
IMAGE_INDX = 20
sample_image_input = digits["images"][IMAGE_INDX].reshape(1, -1)

# Предсказание и отображение цифры
print("Target value:", digits.target[IMAGE_INDX])
print("Probabilities:", model.predict(sample_image_input))
print("Predicted value:", model.predict(sample_image_input).argmax())
plt.imshow(digits.images[IMAGE_INDX])
plt.show()

### Выбор гиперпараметров с `GridSearchCV`

In [ ]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
# Преобразование keras модели в модель sklearn
# Замечание: установите verbose=1, чтобы выводить информацию по обучению и предсказанию
model = KerasClassifier(build_fn=build_model, verbose=0)

# Сетка параметров
parameters = {
    "epochs": [10,],
    "batch_size": [5,],
    "hidden_neurons": [64, 96, 128, 256, 512],
}

# Параметры обучения
grid_class_parameters = {
    "estimator": model,
    "param_grid": parameters,
    "cv": 3,
    "scoring": "neg_log_loss",
    "return_train_score": False
}

# Выбор параметров
grid_search = GridSearchCV(**grid_class_parameters)
grid_search.fit(X_train, Y_train)

# Параметры лучшей модели
print("Parameters of the best model:")
for param, value in grid_search.best_params_.items():
    print("\t{} = {}".format(param, value))

# Лучшая модель
best_model = grid_search.best_estimator_

# Оценка качества распознавания (доля правильных классификаций)
# Замечания: для обучения использовалась neg_log_loss, поэтому, чтобы получить 
#  accuracy, извлекаем модель, для которой ранее указали accuracy в качестве метрики.
train_error = best_model.score(X_train, y_train)
test_error = best_model.score(X_test, y_test)
print("\nTrain Accuracy:", train_error)
print("Test Accuracy:", test_error)

In [ ]:
IMAGE_INDX = 20
sample_image_input = digits["images"][IMAGE_INDX].reshape(1, -1)

# Предсказание и отображение цифры
print("Target value:", digits.target[IMAGE_INDX])
print("Probabilities:", grid_search.predict_proba(sample_image_input))
print("Predicted value:", grid_search.predict(sample_image_input))
plt.imshow(digits.images[IMAGE_INDX])
plt.show()

## Регрессия

In [ ]:
# TODO